### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [3]:
# read the data into pandas dataframes
MN_T24_GD_417a = pd.read_csv("lca/cleaned/417_MN_T24_GD_lca.csv")

# delete any Metazoa hits because they're trypsin
MN_T24_GD_417b = MN_T24_GD_417a[MN_T24_GD_417a.kingdom != 'Metazoa']

# keep only entries to the phylum level
MN_T24_GD_417 = MN_T24_GD_417b[MN_T24_GD_417b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(MN_T24_GD_417))

MN_T24_GD_417.head(6)

# of phylum peptides =  16


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
5,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,SQMLNAAK,Eubacteriales,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,QEFLNAAK,Phorcysia thermohydrogeniphila,Bacteria,NaN,NaN,NaN,Aquificae,NaN,NaN,Aquificae,...,Phorcysia,NaN,NaN,NaN,Phorcysia thermohydrogeniphila,NaN,NaN,NaN,NaN,NaN
164,LSGWGTNK,Rhizopus stolonifer,Eukaryota,Fungi,NaN,NaN,Mucoromycota,Mucoromycotina,NaN,Mucoromycetes,...,Rhizopus,NaN,NaN,NaN,Rhizopus stolonifer,NaN,NaN,NaN,NaN,NaN
174,VAEVSPLR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [5]:
MN_T24_GD_417_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/417A_MN_T24_GD_DN50_ILnaafs.csv")
MN_T24_GD_417_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/417B_MN_T24_GD_DN50_ILnaafs.csv")
#MN_T24_GD_417_NAAFc = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/417C_MN_T24_GD_DN50_ILnaafs.csv")


frames = [MN_T24_GD_417_NAAFa, MN_T24_GD_417_NAAFb]

MN_T24_GD_417_NAAF = pd.concat(frames, sort=False)

MN_T24_GD_417_NAAF.set_index('stripped_peptide')
MN_T24_GD_417_NAAF = MN_T24_GD_417_NAAF.loc[:, ~MN_T24_GD_417_NAAF.columns.str.contains('^Unnamed')]

MN_T24_GD_417_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(MN_T24_GD_417_NAAF))

print('column names:', MN_T24_GD_417_NAAF.columns)

MN_T24_GD_417_NAAF.head()

# of total peptides =  438
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,VLEGNEQFLNAAK,1760000.0,135384.615385
1,VLEGNEQFLNAAK,1760000.0,135384.615385
2,SPATLNSR,415000.0,51875.000000
3,SCK,73800.0,24600.000000
4,SPATLNSR,415000.0,51875.000000


In [6]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(MN_T24_GD_417["peptide"], MN_T24_GD_417_NAAF["peptide"])

print(over)

['APNNLHK' 'LATVLSPR' 'LSGWGTNK' 'NNWGPLK' 'QEFLNAAK' 'VAEVSPLR'
 'VATVSPLR' 'VPTVSPLR']


In [7]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

MN_T24_GD_417.set_index('peptide', inplace=True)
MN_T24_GD_417_NAAF.set_index('peptide', inplace=True)

MN_T24_GD_417_Phy = MN_T24_GD_417.join(MN_T24_GD_417_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(MN_T24_GD_417_Phy))

MN_T24_GD_417_Phy.head()

# of total phylum-level peptides =  39


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
APNNLHK,Bacteroidetes,Bacteria,NaN,NaN,NaN,Bacteroidetes,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20800000.0,2600000.0
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20800000.0,2600000.0
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20800000.0,2600000.0
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20800000.0,2600000.0


In [8]:
# write to a csv

MN_T24_GD_417_Phy.to_csv("lca/NAAF/MN_T24_GD_417_Phy_naaf.csv")